In [1]:
import json
import requests

payload = {
    "@context": [
        "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld",
        {
            "fiware": "https://uri.fiware.org/ns/dataModels#",
            "schema": "https://schema.org/"
        }
    ],
    "id": "urn:ngsi-ld:EVChargingStation:Station002",
    "type": "EVChargingStation",
    "stationName": {
        "type": "Property",
        "value": "CS2 of Drahi-X "
    },
    "capacity": {
        "type": "Property",
        "value": 1
    },
    "availableCapacity": {
        "type": "Property",
        "value": 1
    },
    "allowedVehicleType": {
        "type": "Property",
        "value": ["car", "motorcycle"]
    },
    "socketType": {
        "type": "Property",
        "value": ["Type2", "CHAdeMO"]
    },
    "status": {
        "type": "Property",
        "value": "working"
    },
    "ratedVoltage": {
        "type": "Property",
        "value": 400,
        "unitCode": "VLT"
    },
    "powerConsumption": {
        "type": "Property",
        "value": 22.5,
        "unitCode": "KWT"
    },
    "dateFirstCreat": {
        "type": "Property",
        "value": "2025-06-03T10:30:00Z"
    }
}


response = requests.post(
    url='http://localhost:1026/ngsi-ld/v1/entities',
    headers={"content-type": "application/ld+json"},  
    data=json.dumps(payload)
)

print(f"status_code: {response.status_code}")

status_code: 201


In [12]:
import requests
import json
import time
import random
from datetime import datetime
import threading
from IPython.display import display, clear_output

# Configuration
ORION_URL = "http://localhost:1026"
ENTITY_ID = "urn:ngsi-ld:EVChargingStation:Station002"

class ChargingStationUpdater:
    def __init__(self):
        self.running = False
        self.update_count = 0
        
    def get_current_time(self):
        """Get current timestamp"""
        return datetime.utcnow().isoformat() + "Z"
    
    def simulate_external_api_data(self):
        """Simulate sensor data returned from external API"""
        return {
            "availableCapacity": random.randint(0, 1),  # 0 or 1 charging slot available
            "powerConsumption": round(random.uniform(18.0, 25.0), 2),  # Power consumption variation
            "ratedVoltage": random.choice([380, 400, 420]),  # Voltage fluctuation
        }
    
    def update_station_data(self, new_data):
        """Update charging station data"""
        current_time = self.get_current_time()
        
        # Build update payload - using tested successful format
        update_payload = {}
        for attr_name, value in new_data.items():
            update_payload[attr_name] = {
                "type": "Property",
                "value": value,
                "observedAt": current_time
            }
            
            # Add unit codes
            if attr_name == "powerConsumption":
                update_payload[attr_name]["unitCode"] = "KWT"
            elif attr_name == "ratedVoltage":
                update_payload[attr_name]["unitCode"] = "VLT"
        
        # Send update request - using tested successful configuration
        url = f"{ORION_URL}/ngsi-ld/v1/entities/{ENTITY_ID}/attrs"
        headers = {"content-type": "application/json"}  # Using tested successful content-type
        
        try:
            response = requests.patch(url, headers=headers, data=json.dumps(update_payload))
            return response.status_code, new_data
        except Exception as e:
            return None, str(e)
    
    def get_current_status(self):
        """Get current charging station status"""
        url = f"{ORION_URL}/ngsi-ld/v1/entities/{ENTITY_ID}"
        headers = {"Accept": "application/ld+json"}
        
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.json()
            return None
        except:
            return None
    
    def display_status(self, new_data, status_code):
        """Display update status in Jupyter"""
        clear_output(wait=True)  # Clear previous output
        
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        print("CHARGING STATION REAL-TIME MONITORING")
        print("=" * 60)
        print(f"Time: {current_time}")
        print(f"Update Count: #{self.update_count}")
        print(f"API Status: {'SUCCESS' if status_code == 204 else 'FAILED'} (Status Code: {status_code})")
        
        print("\nLatest Sensor Data:")
        for key, value in new_data.items():
            unit = " slots" if key == "availableCapacity" else " kW" if key == "powerConsumption" else " V"
            print(f"   {key}: {value}{unit}")
        
        # Display current entity complete status
        entity = self.get_current_status()
        if entity:
            print(f"\nCharging Station Overview:")
            print(f"   Name: {entity.get('stationName', {}).get('value', 'N/A')}")
            print(f"   Available Capacity: {entity.get('availableCapacity', {}).get('value', 'N/A')}")
            print(f"   Current Power Consumption: {entity.get('powerConsumption', {}).get('value', 'N/A')} kW")
            print(f"   Rated Voltage: {entity.get('ratedVoltage', {}).get('value', 'N/A')} V")
            print(f"   Status: {entity.get('status', {}).get('value', 'N/A')}")
        
        print(f"\nNext update countdown: 10 seconds...")
        print("To stop updates, interrupt kernel or run updater.stop()")
    
    def start_updating(self, interval=10):
        """Start periodic updates"""
        self.running = True
        self.update_count = 0
        
        print("Starting periodic charging station data updates...")
        print("Simulating external API pushing data every 10 seconds")
        print("To stop, run: updater.stop()")
        
        def update_loop():
            while self.running:
                try:
                    # 1. Simulate getting data from external API
                    sensor_data = self.simulate_external_api_data()
                    
                    # 2. Update NGSI-LD entity
                    status_code, updated_data = self.update_station_data(sensor_data)
                    
                    # 3. Update counter
                    self.update_count += 1
                    
                    # 4. Display status
                    self.display_status(updated_data, status_code)
                    
                    # 5. Wait for next update
                    time.sleep(interval)
                    
                except Exception as e:
                    print(f"ERROR: Error during update process: {e}")
                    self.running = False
        
        # Run update loop in background thread
        self.thread = threading.Thread(target=update_loop)
        self.thread.daemon = True
        self.thread.start()
    
    def stop(self):
        """Stop updates"""
        self.running = False
        print(f"\nUpdates stopped (total {self.update_count} updates executed)")
    
    def single_update_test(self):
        """Execute a single update test"""
        print("Executing single update test...")
        
        sensor_data = self.simulate_external_api_data()
        status_code, updated_data = self.update_station_data(sensor_data)
        
        if status_code == 204:
            print("SUCCESS: Single update successful!")
            print("Updated data:", updated_data)
        else:
            print(f"ERROR: Update failed, status code: {status_code}")

# Create updater instance
updater = ChargingStationUpdater()

print("USAGE:")
print("1. Single test: updater.single_update_test()")
print("2. Start periodic updates: updater.start_updating()")
print("3. Stop updates: updater.stop()")

USAGE:
1. Single test: updater.single_update_test()
2. Start periodic updates: updater.start_updating()
3. Stop updates: updater.stop()


In [13]:
updater.single_update_test()

Executing single update test...
SUCCESS: Single update successful!
Updated data: {'availableCapacity': 0, 'powerConsumption': 22.83, 'ratedVoltage': 420}


In [14]:
updater.start_updating()

CHARGING STATION REAL-TIME MONITORING
Time: 2025-06-04 12:23:25
Update Count: #14
API Status: SUCCESS (Status Code: 204)

Latest Sensor Data:
   availableCapacity: 1 slots
   powerConsumption: 23.73 kW
   ratedVoltage: 400 V

Charging Station Overview:
   Name: CS2 of Drahi-X 
   Available Capacity: 1
   Current Power Consumption: 23.73 kW
   Rated Voltage: 400 V
   Status: working

Next update countdown: 10 seconds...
To stop updates, interrupt kernel or run updater.stop()


In [15]:
updater.stop()


Updates stopped (total 14 updates executed)
